In [42]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=4, app_name='pdu_syp2', node_spec=NODE_MEDIUM)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 24G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp2...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:26


- 성장형 무기 소지자가, 해당 스킨을 안쓰고 다른 스킨을 쓰는 경우가 얼마나 되는지
- 성장형 스킨을 뽑은 사람은 잠시 그것을 쓰겠지만, 철이 지나고 나면 다른 일반 고급스킨으로 갈아타는 패턴이 생겼으면 좋겠다고 생각하는데, 과연 그렇게 흐르는지
- 성장형 무기가 장기적으로 다른 스킨 매출에 영향을 주는지를 유추해볼 수 있지 않을까

In [ ]:
gcoin = load_data_mart(start_date="2021-08-11", end_date="2021-09-07", device="pc", table_name="gcoin_use")

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:46107)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", 

In [59]:
# 10월 성장형 무기 스킨 출시 후 2주간 contraband로 아이템 획득

device = 'pc' 
env = 'live' 
log_name = 'ItemUpgraded'
start_date = '2021-10-06'
end_date = '2021-10-19' 
item_upgrade = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [60]:
# 10월 성장형 무기 스킨 출시 후 4주간 아이템 착용

device = 'pc' 
env = 'live' 
log_name = 'ItemEquipped'
start_date = '2021-10-06'
end_date = '2021-11-02' 
item_equipped = load_schema.lobby(spark, device, env, log_name, start_date, end_date)

In [32]:
item_equipped.select("PartId").distinct().show(truncate=False)

+--------------------------------+
|PartId                          |
+--------------------------------+
|partdesc.crossbow               |
|partdesc.r45:partdesc.charm     |
|partdesc.mg3:partdesc.charm     |
|partdesc.dp_12:partdesc.charm   |
|partdesc.crossbow:partdesc.charm|
|partdesc.m16a4:partdesc.charm   |
|partdesc.scooter                |
|partdesc.skorpion:partdesc.charm|
|partdesc.feet                   |
|partdesc.parachute              |
|partdesc.awm                    |
|partdesc.kar98                  |
|partdesc.win94:partdesc.charm   |
|partdesc.dp_12                  |
|partdesc.sks:partdesc.charm     |
|partdesc.backpacklv2            |
|partdesc.fnfal:partdesc.charm   |
|partdesc.buggy                  |
|partdesc.m762:partdesc.charm    |
|partdesc.dirtbike               |
+--------------------------------+
only showing top 20 rows



In [29]:
item_equipped.select("ItemId").distinct().show(20, truncate=False)

+-----------------+
|ItemId           |
+-----------------+
|itemdesc.18020049|
|itemdesc.11030187|
|itemdesc.12010201|
|itemdesc.12010303|
|itemdesc.12010350|
|itemdesc.11050121|
|itemdesc.11010178|
|itemdesc.11010158|
|itemdesc.41010079|
|itemdesc.11020176|
|itemdesc.31050080|
|itemdesc.11050133|
|itemdesc.11060104|
|itemdesc.12032083|
|itemdesc.12032040|
|itemdesc.11050152|
|itemdesc.11080048|
|itemdesc.11020120|
|itemdesc.11010264|
|itemdesc.12032063|
+-----------------+
only showing top 20 rows



In [49]:
items = mysql.read_table(spark, "metainfo", "item_meta_sales")
wsus_product_id = items.where(lower(col("product_name")).like("%progressive%")).select(col("product_id")).distinct().toPandas()["product_id"].tolist()

In [50]:
wsus_product_id

['itemdesc.12012001',
 'itemdesc.12012004',
 'itemdesc.12012002',
 'itemdesc.12012003']

In [63]:
# 성장형 무기를 획득한 사람들, 
# ItemDescId, AccountId
wsus_account = item_upgrade.where(col("ItemDescId").isin(wsus_product_id)).select("ItemDescId", "AccountId").distinct().toPandas()

In [64]:
wsus_account.shape

(84439, 2)

In [65]:
wsus_account.head()

,ItemDescId,AccountId
0,itemdesc.12012003,account.61258af00a554e16a04378e6135c6fff
1,itemdesc.12012003,account.d05bf8d9d89a41538230d98ebefedc0b
2,itemdesc.12012003,account.dfa65a48d2fd4992b228aedcf91997b4
3,itemdesc.12012003,account.4669517abf6c4395b3152398f1b2a93d
4,itemdesc.12012003,account.2305c845d069455db221cca8a7c45b3c


In [205]:
oct_wsus_account_list = wsus_account[wsus_account.ItemDescId == "itemdesc.12012003"].AccountId.tolist()

In [206]:
len(oct_wsus_account_list)

69630

In [207]:
wsus_account[wsus_account.AccountId.isin(oct_wsus_account_list)].groupby("AccountId", as_index=False)["ItemDescId"].count().describe()

,ItemDescId
count,69630.000000
mean,1.107770
std,0.340046
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,3.000000


In [70]:
# 성장형 무기 보유자들의 성장형 무기 보유 개수
wsus_cnt = item_upgrade.where((col("ItemDescId").isin(wsus_product_id))).groupBy("AccountId").agg(countDistinct("ItemDescId")).toPandas()

In [71]:
wsus_cnt.describe()

,count(DISTINCT ItemDescId)
count,76703.000000
mean,1.100857
std,0.329188
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,3.000000


In [103]:
weapon_product_list = items.where(col("sub_category") == "weapon").select(col("product_id")).distinct().toPandas().product_id.tolist()

In [104]:
tmp = item_equipped.where((col("AccountId").isin(oct_wsus_account_list)) & (col("ItemId").isin(weapon_product_list))).toPandas()

In [114]:
tmp.head()

,date,AccountId,ItemId,PartId,PlatformProvider,Provider,Time,Type
0,2021-10-06,account.7d67dad8aa2d4aae8c1f11cb07d49b80,itemdesc.12012003,partdesc.m416,PcSteam,steam,2021-10-06T12:38:43.7824946Z,ItemEquipped
1,2021-10-06,account.3ff155d0d32f43769f31b896c3069b72,itemdesc.12012003,partdesc.m416,PcSteam,steam,2021-10-06T12:38:59.8218576Z,ItemEquipped
2,2021-10-06,account.e5005c71058d4ab4bb97fb556dd10186,itemdesc.12010441,partdesc.akm,PcSteam,steam,2021-10-06T12:39:11.2401837Z,ItemEquipped
3,2021-10-06,account.7e84e8935f764f7c98b5f41974840f08,itemdesc.12012003,partdesc.m416,PcSteam,steam,2021-10-06T12:39:31.5596826Z,ItemEquipped
4,2021-10-06,account.421826650db54181ae7b95d12d008db2,itemdesc.12012003,partdesc.m416,PcSteam,steam,2021-10-06T12:40:12.5813152Z,ItemEquipped


In [208]:
oct_wsus_equip_account_list = tmp[(tmp.AccountId.isin(oct_wsus_account_list)) & (tmp.ItemId == "itemdesc.12012003")].AccountId.unique().tolist()

In [209]:
len(oct_wsus_equip_account_list)

68800

68800명 대하여 기간동안 사용한 무기 스킨 수

In [210]:
tmp[tmp.AccountId.isin(oct_wsus_equip_account_list)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,68800.000000
mean,2.196890
std,2.282505
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,44.000000


In [211]:
tmp.PartId.unique()

array(['partdesc.m416', 'partdesc.akm', 'partdesc.fnfal', 'partdesc.g36c',
       'partdesc.m762', 'partdesc.m24', 'partdesc.kar98',
       'partdesc.sawed_off', 'partdesc.sks', 'partdesc.mosin_nagant',
       'partdesc.vector', 'partdesc.mini14', 'partdesc.pan',
       'partdesc.qbz95', 'partdesc.aug_a3', 'partdesc.mk14',
       'partdesc.p18c', 'partdesc.win94', 'partdesc.qbu88',
       'partdesc.groza', 'partdesc.s1897', 'partdesc.crowbar',
       'partdesc.scal_l', 'partdesc.machete', 'partdesc.m16a4',
       'partdesc.ump9', 'partdesc.vss', 'partdesc.mk47',
       'partdesc.bizonpp19', 'partdesc.micro_uzi', 'partdesc.skorpion',
       'partdesc.p92', 'partdesc.m249', 'partdesc.awm', 'partdesc.dp_28',
       'partdesc.sickle', 'partdesc.s12k', 'partdesc.crossbow',
       'partdesc.s686', 'partdesc.r1895'], dtype=object)

In [212]:
# 10월 성장형 무기 소유자 & 이용자(68800)에 대한 무기에 대해 소비
gcoin_use = gcoin.where((col("account_id").isin(oct_wsus_equip_account_list)) & (col("product_id").isin(weapon_product_list))).groupBy("account_id", "product_id").agg((sum("paid_use")/100).alias("paid")).toPandas()

그래프 용
- 일별 사용 무기 스킨
- 무기 스킨별 매출
- m416 무기 스킨별 매출

In [257]:
# mysql.drop_table("labs", "oct_wsus_item_equipped_table")

In [258]:
oct_wsus_item_equipped = item_equipped.where((col("AccountId").isin(oct_wsus_account_list)) & (col("ItemId").isin(weapon_product_list))).groupBy("date", "AccountId", "ItemId").agg(count("*").alias("cnt"))
product_name = items.select("product_id", "product_name").distinct()
oct_wsus_item_equipped_table = oct_wsus_item_equipped.join(product_name, oct_wsus_item_equipped_table.ItemId == product_name.product_id, "left")
mysql.insert_table(oct_wsus_item_equipped_table, "labs", "oct_wsus_item_equipped_table")

In [259]:
oct_wsus_item_equipped_table.printSchema()

root
 |-- date: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- ItemId: string (nullable = true)
 |-- cnt: long (nullable = false)
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)



In [260]:
oct_wsus_item_equipped_table.groupBy("product_name").agg(sum("cnt").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,[PROGRESSIVE] Trick-or-treat - M416,121755
1,[PROGRESSIVE] Buzzkill - SLR,8527
2,[PROGRESSIVE] Gear Head - Beryl M762,8402
3,Gold Plate - Beryl M762,6735
4,Bunny Aeronautics - M416,5419
5,White Rabbit - Pan,4805
6,Silver Plate - Beryl M762,4048
7,Silver Plate - M416,3477
8,CNH's SLR,2378
9,Bunny Aeronautics - Mini14,2283


In [261]:
oct_wsus_item_equipped_table.groupBy("product_name").agg(countDistinct("AccountId").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,[PROGRESSIVE] Trick-or-treat - M416,68800
1,[PROGRESSIVE] Buzzkill - SLR,6026
2,[PROGRESSIVE] Gear Head - Beryl M762,4439
3,Gold Plate - Beryl M762,4144
4,White Rabbit - Pan,3841
5,Bunny Aeronautics - M416,2954
6,Silver Plate - Beryl M762,2101
7,Molten Mayhem - M249,2027
8,Bunny Aeronautics - Mini14,1947
9,Night Slayer - Crossbow,1778


In [249]:
oct_wsus_gcoin_table = gcoin.where((col("account_id").isin(oct_wsus_equip_account_list)) & (col("product_id").isin(weapon_product_list))).select("date", "account_id", "product_id", "free_use", "paid_use", "currency", "product_name")
mysql.insert_table(oct_wsus_gcoin_table, "labs", "oct_wsus_gcoin_table")

In [262]:
oct_wsus_gcoin_table.printSchema()

root
 |-- date: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- free_use: long (nullable = true)
 |-- paid_use: long (nullable = true)
 |-- currency: string (nullable = true)
 |-- product_name: string (nullable = true)



In [263]:
oct_wsus_gcoin_table.groupBy("product_name").agg((sum("paid_use")/100).alias("revenue")).orderBy("revenue", ascending=False).toPandas().head(20)

,product_name,revenue
0,White Rabbit - Pan,5983.2
1,LongSkr's AKM,2591.4
2,Molten Mayhem - M249,2171.8
3,Bunny Aeronautics - M416,2170.5
4,Manticore - SLR,2017.8
5,Bunny Aeronautics - Mini14,1689.7
6,Harlequin - SLR,1289.3
7,Gold Plate - Beryl M762,1155.9
8,Bunnycomb - M24,1070.2
9,Son Heung-min - M416,1034.2


In [253]:
oct_m416_gcoin_table = gcoin.where((col("account_id").isin(oct_wsus_account_list)) & (col("product_id").isin(m416_skin_list))).select("date", "account_id", "product_id", "free_use", "paid_use", "currency", "product_name")
mysql.insert_table(oct_m416_gcoin_table, "labs", "oct_m416_gcoin_table")

In [264]:
oct_m416_gcoin_table.groupBy("product_name").agg((sum("paid_use")/100).alias("revenue")).orderBy("revenue", ascending=False).toPandas().head(20)

,product_name,revenue
0,Bunny Aeronautics - M416,2211.5
1,Son Heung-min - M416,1063.3


In [265]:
oct_m416_gcoin_table.groupBy("product_name").agg(countDistinct("account_id").alias("cnt")).orderBy("cnt", ascending=False).toPandas().head(20)

,product_name,cnt
0,Bunny Aeronautics - M416,339
1,Son Heung-min - M416,139


In [213]:
tmp2 = pd.merge(tmp, gcoin_use, left_on=["AccountId", "ItemId"], right_on=["account_id", "product_id"], how="left").drop(["PlatformProvider", "Provider", "Type", "account_id", "product_id"], axis=1)

In [214]:
# 10개 초과 unique 무기 스킨 장착한 대상
top_tmp = tmp[tmp.AccountId.isin(oct_wsus_equip_account_list)].groupby("AccountId")["ItemId"].nunique().reset_index()
top_account_list = top_tmp[top_tmp.ItemId > 10].AccountId.tolist()

In [215]:
len(top_account_list)

902

In [216]:
skin_buy_and_equip_account_top_list = tmp2[tmp2.AccountId.isin(top_account_list)].dropna().AccountId.unique().tolist()
len(skin_buy_and_equip_account_top_list)

179

In [217]:
skin_buy_and_equip_account_list = tmp2.dropna().AccountId.unique().tolist()
len(skin_buy_and_equip_account_list)

6502

In [218]:
tmp2[tmp2.AccountId.isin(skin_buy_and_equip_account_list)].groupby(["date", "AccountId", "ItemId", "PartId"], as_index=False)["paid"].sum().sort_values(["AccountId", "date"]).to_csv("./data/skin_buy.csv", index=False)

10월 성장형 무기 사용한 사람들 대상 (oct_wsus_equip_account_list) m416무기에 대한 스킨 착용

In [219]:
m416 = tmp2[(tmp2.AccountId.isin(oct_wsus_equip_account_list)) & (tmp2.PartId == "partdesc.m416")].sort_values(["AccountId", "date"]).groupby(["date", "AccountId", "ItemId"], as_index=False)["paid"].sum().sort_values(["AccountId", "date"])

In [252]:
m416_skin_list = m416.ItemId.unique().tolist()

In [220]:
m416.groupby(["AccountId"])["ItemId"].nunique().reset_index().describe()

,ItemId
count,68800.000000
mean,1.145799
std,0.467780
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,8.000000


In [221]:
m416.groupby(["AccountId"])["ItemId"].nunique().reset_index().groupby("ItemId", as_index=False)["AccountId"].count()

,ItemId,AccountId
0,1,61031
1,2,6158
2,3,1180
3,4,289
4,5,90
5,6,32
6,7,14
7,8,6


In [232]:
90+32+14+6

142

m416 스킨 1개 사용한 사람은 12012003만 사용한게 맞는지 확인

In [222]:
a = m416.groupby(["AccountId"])["ItemId"].nunique().reset_index()

In [223]:
m416[m416.AccountId.isin(a[a.ItemId == 1].AccountId.unique().tolist())].ItemId.unique()

array(['itemdesc.12012003'], dtype=object)

In [225]:
mskin_cnt = m416.groupby(["AccountId"])["ItemId"].nunique().reset_index()

In [227]:
mskin2 = mskin_cnt[mskin_cnt.ItemId == 2].AccountId.unique().tolist()
mskin3 = mskin_cnt[mskin_cnt.ItemId == 3].AccountId.unique().tolist()
mskin4 = mskin_cnt[mskin_cnt.ItemId == 4].AccountId.unique().tolist()
mskin5 = mskin_cnt[mskin_cnt.ItemId >= 5].AccountId.unique().tolist()

In [231]:
m416[(m416.AccountId.isin(mskin2)) & (m416.paid>0)].AccountId.nunique()

305

In [236]:
m416[(m416.AccountId.isin(mskin2)) & (m416.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,305.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [242]:
m416[(m416.AccountId.isin(mskin2)) & (m416.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010445     76
itemdesc.12010477    229
Name: AccountId, dtype: int64

In [233]:
m416[(m416.AccountId.isin(mskin3)) & (m416.paid>0)].AccountId.nunique()

41

In [238]:
m416[(m416.AccountId.isin(mskin3)) & (m416.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,41.000000
mean,1.073171
std,0.263652
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [241]:
m416[(m416.AccountId.isin(mskin3)) & (m416.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010445    13
itemdesc.12010477    31
Name: AccountId, dtype: int64

In [234]:
m416[(m416.AccountId.isin(mskin4)) & (m416.paid>0)].AccountId.nunique()

8

In [239]:
m416[(m416.AccountId.isin(mskin4)) & (m416.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,8.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [243]:
m416[(m416.AccountId.isin(mskin4)) & (m416.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010445    4
itemdesc.12010477    4
Name: AccountId, dtype: int64

In [235]:
m416[(m416.AccountId.isin(mskin5)) & (m416.paid>0)].AccountId.nunique()

7

In [244]:
m416[(m416.AccountId.isin(mskin5)) & (m416.paid>0)].groupby("ItemId")["AccountId"].nunique()

ItemId
itemdesc.12010445    4
itemdesc.12010477    4
Name: AccountId, dtype: int64

In [240]:
m416[(m416.AccountId.isin(mskin5)) & (m416.paid>0)].groupby("AccountId")["ItemId"].nunique().reset_index().describe()

,ItemId
count,7.000000
mean,1.142857
std,0.377964
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,2.000000


In [4]:
7/52*100

13.461538461538462

In [5]:
248+56

304